In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

df = pd.read_csv(r'C:\Data\NVDA.csv')

window = 20
df['MA'] = df['Close'].rolling(window=window).mean()
df['STD'] = df['Close'].rolling(window=window).std()
df['UpperBand'] = df['MA'] + (2 * df['STD'])
df['LowerBand'] = df['MA'] - (2 * df['STD'])

delta = df['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
rs = gain / loss
df['RSI'] = 100 - (100 / (1 + rs))
df['Volume_MA'] = df['Volume'].rolling(window=window).mean()
df['Volume_STD'] = df['Volume'].rolling(window=window).std()
df['Volatility'] = df['Close'].rolling(window=window).std()

features = ['Close', 'UpperBand', 'LowerBand', 'RSI', 'Volume_MA', 'Volume_STD', 'Volatility']
target = 'Close'

df['Target'] = df['Close'].shift(-1)
df.dropna(inplace=True)

scaler = MinMaxScaler(feature_range=(0, 1))
df[features] = scaler.fit_transform(df[features])


def create_sequences(data, seq_length):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        sequences.append(data.iloc[i:i + seq_length][features].values)
        targets.append(data.iloc[i + seq_length][target])
    return np.array(sequences), np.array(targets)


sequence_length = 10

X, y = create_sequences(df, sequence_length)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(LSTM(units=32, return_sequences=True, input_shape=(X_train.shape[1], len(features))))
model.add(Dropout(0.2))
model.add(LSTM(units=32))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE):", rmse)

plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(y_test, label='Actual Closing Prices', color='blue')
plt.plot(y_pred, label='Predicted Closing Prices', color='red')
plt.title('Actual vs Predicted Closing Prices')
plt.xlabel('Time')
plt.ylabel('Closing Price')
plt.legend()
plt.show()
